In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.io import loadmat
from scipy import stats

import sys
import matplotlib.pyplot as plt

from return_period import percentile_data, return_period, return_period_ensemble

In [ ]:

### Truth

# Load the data

DATA_DIR = '/ocean/projects/phy220045p/jakhar/py2d_dealias/postprocess/results_aposteriori/Re500_fkx4fky4_r0.1_b20/'
filename = 'omega_extreme_DNS_NX64_dt0.0005_IC1'
dt_data = 2e-2


# ## Ground Truth

Omega_max_arr = []
Omega_min_arr = []

for ic in range(1,11):
    filename = f'omega_extreme_DNS_NX64_dt0.0005_IC{ic}.mat'
    data = loadmat(DATA_DIR + '/' + filename)

    Omega_max_arr.append(data['Omega_max'])
    Omega_min_arr.append(data['Omega_min'])

    omega_mean = data['Omega_mean']
    omega_std = data['Omega_std']

    print(data['Omega_mean'], data['Omega_std'], np.asarray(data['Omega_max']).shape, np.asarray(data['Omega_min']).shape)

omega_max = np.asarray(Omega_max_arr).flatten()
omega_min = np.asarray(Omega_min_arr).flatten()

omega_max_normalized = omega_max / omega_std[0][0]
omega_min_normalized = omega_min / omega_std[0][0]

# desired_length = 25000000
# while len(omega_max_normalized) > desired_length:
#     omega_max_normalized = omega_max_normalized[:desired_length]
#     omega_min_normalized = omega_min_normalized[:desired_length]

split_size = 100
omega_max_normalized_split = np.split(omega_max_normalized, split_size)
omega_min_normalized_split = np.split(omega_min_normalized, split_size)

In [ ]:
def percentile_data(data, percentile):
    """
    Calculate error bands and
    return the lower/upper bounds in percentile

    Parameters:
    -----------
    data : np.ndarray
        2D NumPy array of shape (N, samples)
    percentile : float
        A number between 0 and 100 (typically <= 50 for symmetrical bounds)
    
    Returns:
    --------
    means : np.ndarray
        Array of sample means (shape: N)
    lower_bounds: np.ndarray
        Array of lower bounds in percentage relative to the mean (shape: N)
    upper_bounds: np.ndarray
        Array of upper bounds in percentage relative to the mean (shape: N)
    """
    # Mean of each row
    means = np.mean(data, axis=0)
    
    # Lower (p-th) and upper ((100-p)-th) percentiles of each row
    lower_vals = np.percentile(data, percentile, axis=0)
    upper_vals = np.percentile(data, 100 - percentile, axis=0)

    print(data.shape)
    # print(lower_vals.shape, upper_vals.shape)
    # print(lower_vals, upper_vals)
    
    # Calculate percentage difference relative to the mean
    # (m - L)/m * 100 for lower, (U - m)/m * 100 for upper
    # lower_bounds = 100.0 * (means - lower_vals) / means
    # upper_bounds = 100.0 * (upper_vals - means) / means
    
    return means, lower_vals, upper_vals

def std_dev_data(data, std_dev=1):
    """
    Calculate error bands and
    return the lower/upper bounds in percentile

    Parameters:
    -----------
    data : np.ndarray
        2D NumPy array of shape (N, samples)
    std_dev : float
        A number between 0 and 100 (typically <= 50 for symmetrical bounds)
    
    Returns:
    --------
    means : np.ndarray
        Array of sample means (shape: N)
    lower_bounds: np.ndarray
        Array of lower bounds in percentage relative to the mean (shape: N)
    upper_bounds: np.ndarray
        Array of upper bounds in percentage relative to the mean (shape: N)
    """
    # Mean of each row
    means = np.mean(data, axis=0)
    stds = np.std(data, axis=0)
    
    # Lower (p-th) and upper ((100-p)-th) percentiles of each row
    lower_vals = means - stds*std_dev
    upper_vals = means + stds*std_dev
    
    return means, lower_vals, upper_vals

def return_period_ensemble(data, dt=1, bins=50, uncertainty='freq_exceedance',
                           confidence_level=25):
    '''
    Calculate return period using ensemble of data
    data: 2D array of data [ensemble, time]
    dt: time step
    bins: number of bins for binning the data
    uncertainty: 'freq' / 'freq_exceedance': 
        Determines whether uncertainty is calculated based on frequency or frequency exceedance
    confidence_level: Percentage for the confidence interval (e.g., 95 for 95% confidence interval)
    '''
    number_ensemble = data.shape[0]
    total_data_points = data.shape[1]
    
    bin_min = np.min(data)
    bin_max = np.max(data)
    
    # Initialize lists to store per-ensemble probabilities
    prob_exceedance_ensemble = []
    freq_ensemble = []

    bins_centers = None  # Will be set during the first iteration

    for i in range(number_ensemble):
        data_ensemble = data[i, :]
        freq_sample, bin_edges = np.histogram(data_ensemble, bins=bins, range=(bin_min, bin_max))
        if bins_centers is None:
            bins_centers = (bin_edges[1:] + bin_edges[:-1]) / 2
        freq_exceedance_sample = np.cumsum(freq_sample[::-1])[::-1]
        prob_exceedance_sample = freq_exceedance_sample / total_data_points

        freq_ensemble.append(freq_sample)
        prob_exceedance_ensemble.append(prob_exceedance_sample)

    freq_ensemble = np.array(freq_ensemble)
    prob_exceedance_ensemble = np.array(prob_exceedance_ensemble)

    if uncertainty == 'freq' or uncertainty == 'freq_exceedance':
        # Calculate uncertainty based on the selected method
        if uncertainty == 'freq':
            # Average frequencies and compute probabilities from the mean frequencies
            freq_mean = np.mean(freq_ensemble, axis=0)

            # Compute cumulative frequencies
            freq_exceedance_mean = np.cumsum(freq_mean[::-1])[::-1]

            # Calculate exceedance probabilities
            prob_exceedance_mean = freq_exceedance_mean / (total_data_points * number_ensemble)

            # Calculate confidence intervals for frequencies
            lower_percentile = (100 - confidence_level) / 2
            upper_percentile = 100 - lower_percentile

            freq_exceedance_lower = np.percentile(np.cumsum(freq_ensemble[:, ::-1], axis=1)[:, ::-1],
                                                lower_percentile, axis=0)
            freq_exceedance_upper = np.percentile(np.cumsum(freq_ensemble[:, ::-1], axis=1)[:, ::-1],
                                                upper_percentile, axis=0)

            # Calculate exceedance probabilities for confidence intervals
            prob_exceedance_lower = freq_exceedance_lower / (total_data_points)
            prob_exceedance_upper = freq_exceedance_upper / (total_data_points)

        elif uncertainty == 'freq_exceedance':
            # Calculate mean of probabilities directly
            # prob_exceedance_mean, prob_exceedance_lower, prob_exceedance_upper = ci_t_distribution(prob_exceedance_ensemble, confidence_level)
            # prob_exceedance_mean, prob_exceedance_lower, prob_exceedance_upper = ci_normal_distribution(prob_exceedance_ensemble, confidence_level)
            # prob_exceedance_mean, prob_exceedance_lower, prob_exceedance_upper = percentile_data(prob_exceedance_ensemble, confidence_level)
            prob_exceedance_mean, prob_exceedance_lower, prob_exceedance_upper = std_dev_data(prob_exceedance_ensemble, 1)

    
        # print(prob_exceedance_mean)
        # print(prob_exceedance_lower)
        # print(prob_exceedance_upper)

        # Ensure probabilities are within valid range
        x = np.clip(prob_exceedance_mean, 1e-14, 1)
        prob_exceedance_lower = np.clip(prob_exceedance_lower, 1e-14, 1)
        prob_exceedance_upper = np.clip(prob_exceedance_upper, 1e-14, 1)

        # Calculate return periods
        return_period_mean = dt / prob_exceedance_mean
        return_period_lower = dt / prob_exceedance_upper  # Higher probability, lower return period
        return_period_upper = dt / prob_exceedance_lower  # Lower probability, higher return period

    elif uncertainty == 'return_period':

        prob_exceedance_ensemble = np.clip(prob_exceedance_ensemble, 1e-14, 1)
        return_period_ensemble = dt/prob_exceedance_ensemble

        return_period_mean, return_period_lower, return_period_upper = percentile_data(return_period_ensemble, confidence_level)
        # return_period_mean, return_period_lower, return_period_upper = std_dev_data(return_period_ensemble, 1)

    else:
        raise ValueError("Invalid uncertainty method. Choose 'freq' or 'freq_exceedance'.")

    return return_period_mean, return_period_lower, return_period_upper, bins_centers



In [ ]:
# Training data
start=5000
omega_max_train_normalized = omega_max_normalized[start:start+2500] 
omega_min_train_normalized = omega_min_normalized[start:start+2500] 

markersize = 1
bin_num = 100
fig, axes = plt.subplots(1, 2, figsize=(10, 5))


### Omega min

return_period_kerry, freq_tot, bins_min = return_period(np.abs(omega_min_train_normalized), dt=dt_data, bins=bin_num)
axes[0].semilogy(-bins_min, return_period_kerry, 'or', label='Train (2500 snapshots)', markersize=markersize)

return_period_mean, return_period_min, return_period_max, bins = return_period_ensemble(
    np.asarray(omega_max_normalized_split), dt=dt_data, bins=bin_num, uncertainty='return_period')
axes[0].semilogy(-bins, return_period_mean, 'ok', label='Truth: 250,000 snapshots; 100 ensembles; 25/75 percentile', markersize=markersize)
axes[0].fill_between(-bins, return_period_min, return_period_max, color='k', alpha=0.2)

axes[0].set_xlabel('$\omega$ min')
axes[0].set_ylabel('Return Period')

# ### Omega max

return_period_kerry, freq_tot, bins_max = return_period(np.abs(omega_max_train_normalized), dt=dt_data, bins=bin_num)
axes[1].semilogy(bins_max, return_period_kerry, 'or', label='Train (2500 snapshots)', markersize=markersize)

return_period_mean, return_period_min, return_period_max, bins = return_period_ensemble(
    np.asarray(omega_max_normalized_split), dt=dt_data, bins=bin_num, uncertainty='return_period')
axes[1].semilogy(bins, return_period_mean, 'ok', label='Truth: 25/75 percentile; 100 ensembles; 250,000 snapshots', markersize=markersize)
axes[1].fill_between(bins, return_period_min, return_period_max, color='k', alpha=0.2)

axes[1].set_xlabel('$\omega$ max')

for ax in axes.flatten():
    ax.set_ylim([1e-2, 1e3])
axes[0].set_xlim([-4.5, -1.7])
axes[1].set_xlim([1.7, 4.5])
# plt.suptitle('97.5% CI error calculated with 200 ensembles of 2500 snapshots')
axes[1].legend(loc='upper left', frameon=False)

plt.show()

In [ ]:
# Training data
start=5000
omega_max_train_normalized = omega_max_normalized[start:start+2500] 
omega_min_train_normalized = omega_min_normalized[start:start+2500] 

markersize = 1
bin_num = 50
fig, axes = plt.subplots(1, 2, figsize=(10, 5))


### Omega min

return_period_kerry, freq_tot, bins_min = return_period(np.abs(omega_min_train_normalized), dt=dt_data, bins=bin_num)
axes[0].semilogy(-bins_min, return_period_kerry, 'or', label='Train (2500 snapshots)', markersize=markersize)

return_period_mean, return_period_min, return_period_max, bins = return_period_ensemble(
    np.asarray(omega_max_normalized_split), dt=dt_data, bins=bin_num, uncertainty='freq_exceedance')
axes[0].semilogy(-bins, return_period_mean, 'ok', label='Truth: 25/75 percentile; 100 ensembles; 250,000 snapshots', markersize=markersize)
axes[0].fill_between(-bins, return_period_min, return_period_max, color='k', alpha=0.2)

axes[0].set_xlabel('$\omega$ min')
axes[0].set_ylabel('Return Period')

# ### Omega max

return_period_kerry, freq_tot, bins_max = return_period(np.abs(omega_max_train_normalized), dt=dt_data, bins=bin_num)
axes[1].semilogy(bins_max, return_period_kerry, 'or', label='Train (2500 snapshots)', markersize=markersize)

return_period_mean, return_period_min, return_period_max, bins = return_period_ensemble(
    np.asarray(omega_max_normalized_split), dt=dt_data, bins=bin_num, uncertainty='freq_exceedance')
axes[1].semilogy(bins, return_period_mean, 'ok', label='Truth: 25/75 percentile; 100 ensembles; 250,000 snapshots', markersize=markersize)
axes[1].fill_between(bins, return_period_min, return_period_max, color='k', alpha=0.2)

axes[1].set_xlabel('$\omega$ max')

for ax in axes.flatten():
    ax.set_ylim([1e-2, 1e6])
# axes[0].set_xlim([-3, -2])
axes[1].set_xlim([1.7, 5])
plt.suptitle('97.5% CI error calculated with 200 ensembles of 2500 snapshots')
axes[1].legend(loc='upper left', frameon=False)

plt.show()

In [ ]:
bin_num = 50
# for count in range(omega_max_normalized_split.shape[0]):
for count in range(len(omega_max_normalized_split)):
    return_period_kerry, freq_tot, bins_max = return_period(np.abs(omega_max_normalized_split[count]), dt=dt_data, bins=bin_num)
    plt.semilogy(bins_max, return_period_kerry, alpha = 0.1)

return_period_mean, return_period_min, return_period_max, bins = return_period_ensemble(
    np.asarray(omega_max_normalized_split), dt=dt_data, bins=bin_num, uncertainty='freq_exceedance')
plt.semilogy(bins_max, return_period_kerry, 'k')




plt.show()
    

In [ ]:
bin_num = 50
# for count in range(omega_max_normalized_split.shape[0]):
for count in range(len(omega_max_normalized_split)):
    return_period_kerry, freq_tot, bins_max = return_period(np.abs(omega_max_normalized_split[count]), dt=dt_data, bins=bin_num)
    plt.semilogy(bins_max, return_period_kerry, alpha = 0.1)

return_period_mean, return_period_min, return_period_max, bins = return_period_ensemble(
    np.asarray(omega_max_normalized_split), dt=dt_data, bins=bin_num, uncertainty='freq_exceedance')
plt.semilogy(bins_max, return_period_kerry, 'k')

for count in range(100):
    start = np.random.randint(5000,25025000)
    omega_max_train_normalized = omega_max_normalized[start:start+25000] 
    return_period_kerry, freq_tot, bins_max = return_period(np.abs(omega_max_train_normalized), dt=dt_data, bins=bin_num)
    plt.semilogy(bins_max, return_period_kerry, color='k', alpha = 0.2)


for count in range(100):
    start = np.random.randint(5000,25005000)
    omega_max_train_normalized = omega_max_normalized[start:start+2500] 
    return_period_kerry, freq_tot, bins_max = return_period(np.abs(omega_max_train_normalized), dt=dt_data, bins=bin_num)
    plt.semilogy(bins_max, return_period_kerry, color='r', alpha = 0.2)

plt.show()

In [ ]:
import sys

dt = 1
bins = 50
confidence_level = 25
uncertainity = 'return_period'

data = np.random.rand(10, 500)

number_ensemble = data.shape[0]
total_data_points = data.shape[1]

bin_min = np.min(data)
bin_max = np.max(data)

# Initialize lists to store per-ensemble probabilities
prob_exceedance_ensemble = []
freq_ensemble = []

bins_centers = None  # Will be set during the first iteration

for i in range(number_ensemble):
    data_ensemble = data[i, :]
    freq_sample, bin_edges = np.histogram(data_ensemble, bins=bins, range=(bin_min, bin_max))
    if bins_centers is None:
        bins_centers = (bin_edges[1:] + bin_edges[:-1]) / 2
    freq_exceedance_sample = np.cumsum(freq_sample[::-1])[::-1]
    prob_exceedance_sample = freq_exceedance_sample / total_data_points

    freq_ensemble.append(freq_sample)
    prob_exceedance_ensemble.append(prob_exceedance_sample)

    print(prob_exceedance_sample.shape)

freq_ensemble = np.array(freq_ensemble)
prob_exceedance_ensemble = np.array(prob_exceedance_ensemble)

prob_exceedance_ensemble = np.clip(prob_exceedance_ensemble, 1e-14, 1)

return_period_ensemble = dt/prob_exceedance_ensemble
prob_exceedance_mean, prob_exceedance_lower, prob_exceedance_upper = std_dev_data(prob_exceedance_ensemble, 1)

            # prob_exceedance_mean, prob_exceedance_lower, prob_exceedance_upper = std_dev_data(prob_exceedance_ensemble, 1)

    
        # print(prob_exceedance_mean)
        # print(prob_exceedance_lower)
        # print(prob_exceedance_upper)

        # # Ensure probabilities are within valid range
        # x = np.clip(prob_exceedance_mean, 1e-14, 1)
        # prob_exceedance_lower = np.clip(prob_exceedance_lower, 1e-14, 1)
        # prob_exceedance_upper = np.clip(prob_exceedance_upper, 1e-14, 1)

        # # Calculate return periods
        # return_period_mean = dt / prob_exceedance_mean
        # return_period_lower = dt / prob_exceedance_upper  # Higher probability, lower return period
        # return_period_upper = dt / prob_exceedance_lower  # Lower probability, higher return period

        # return return_period_mean, return_period_lower, return_period_upper, bins_centers



# return_period_ensemble = np.zeros((number_ensemble, bins))

